# MySQL Mock Data Generation

This notebook generates mock data for `Product` and `Product_Variant` tables and inserts it into a MySQL database.

**Target Table Schemas:**
```sql
CREATE TABLE IF NOT EXISTS Product (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100),
    description TEXT,
    category VARCHAR(50),
    created_at DATETIME
);

CREATE TABLE IF NOT EXISTS Product_Variant (
    id INT PRIMARY KEY AUTO_INCREMENT,
    product_id INT,
    color VARCHAR(50),
    size VARCHAR(50),
    price DECIMAL(10, 2),
    stock_qty INT,
    FOREIGN KEY (product_id) REFERENCES Product(id)
);
```

## 1. Prerequisites

1.  **MySQL Server Running:** Ensure your MySQL server is active.
2.  **Database and Tables Exist:** Create your database and the `Product` and `Product_Variant` tables using the SQL schema above.
3.  **Install Python Libraries:** If you haven't already, install the necessary libraries. Run the following command in your terminal or a code cell with `!pip`:
    ```bash
    pip install mysql-connector-python Faker
    ```

In [2]:
# You can run this cell if you need to install the libraries from within the notebook
!pip install mysql-connector-python Faker


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Import Libraries

In [3]:
import mysql.connector
from faker import Faker
import random
from datetime import datetime, timedelta

## 3. Configuration

**IMPORTANT:** Update `DB_CONFIG` with your MySQL connection details.
Adjust `NUM_PRODUCTS_TO_GENERATE` and `MAX_VARIANTS_PER_PRODUCT` as needed.

In [11]:
# --- Database Configuration --- 
# !!! UPDATE THESE DETAILS !!!
DB_CONFIG = {
    'host': 'localhost',        # Your MySQL host (e.g., 'localhost' or an IP address)
    'user': 'root',        # Your MySQL username
    'password': '',    # Your MySQL password
    'database': 'test_database'   # The name of your database
}

# --- Data Generation Parameters ---
NUM_PRODUCTS_TO_GENERATE = 100  # How many parent products to create
MAX_VARIANTS_PER_PRODUCT = 5   # Max number of variants for each product (min is 1)
BATCH_SIZE = 50                # How many records to insert in one go (for performance)

# --- Faker and Helper Data ---
fake = Faker()

PRODUCT_CATEGORIES = [
    "Electronics", "Clothing", "Books", "Home Goods", "Beauty",
    "Sports & Outdoors", "Toys & Games", "Food & Beverage", "Automotive",
    "Health & Wellness", "Jewelry", "Software", "Garden", "Pet Supplies"
]

COLORS = [
    "Red", "Blue", "Green", "Black", "White", "Silver", "Gold", "Pink", 
    "Purple", "Yellow", "Orange", "Brown", "Gray", "Beige", "Navy"
]

SIZES = [
    "XS", "S", "M", "L", "XL", "XXL", "One Size", "US 6", "US 7", 
    "US 8", "US 9", "US 10", "US 11", "30x30", "32x32", "34x34", "50ml", "100ml"
]

## 4. Data Generation and Insertion Function

In [13]:
def generate_and_insert_mock_data():
    """
    Connects to MySQL, generates mock product and product variant data,
    and inserts it into the database.
    """
    cnx = None
    cursor = None
    total_products_actually_inserted = 0
    total_variants_actually_inserted = 0

    try:
        # Establish database connection
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()
        print(f"Successfully connected to the database: {DB_CONFIG['database']}")

        # --- 1. Generate and Insert Products ---
        print(f"\nGenerating and inserting {NUM_PRODUCTS_TO_GENERATE} products...")
        product_data_batch = []
        inserted_product_ids = [] # To store IDs of inserted products for variants

        sql_product = "INSERT INTO Product (name, description, category, created_at) VALUES (%s, %s, %s, %s)"

        for i in range(NUM_PRODUCTS_TO_GENERATE):
            try:
                name = fake.unique.word().capitalize() + " " + fake.company_suffix() + " " + fake.bs().title()
                if len(name) > 99: name = name[:99] # Ensure name fits VARCHAR(100)
            except: # In case unique runs out for very small generation sets
                name = fake.word().capitalize() + " " + fake.company_suffix() + " " + fake.bs().title()
                if len(name) > 99: name = name[:99]
            
            description = fake.paragraph(nb_sentences=random.randint(2, 5), variable_nb_sentences=True)
            category = random.choice(PRODUCT_CATEGORIES)
            created_at = fake.date_time_between(start_date='-3y', end_date='now')

            product_data_batch.append((name, description, category, created_at))

            if (i + 1) % BATCH_SIZE == 0 or (i + 1) == NUM_PRODUCTS_TO_GENERATE:
                print(f"  Inserting batch of {len(product_data_batch)} products (total attempted: {i+1}/{NUM_PRODUCTS_TO_GENERATE})...")
                cursor.executemany(sql_product, product_data_batch)
                cnx.commit()
                # To get IDs for products just inserted in this batch
                # This is a bit simplified; for very high concurrency, other methods might be needed
                # For this script's purpose, getting IDs after all products or in larger chunks is fine.
                product_data_batch = [] # Clear the batch
        
        print("All products batches sent. Fetching all inserted product IDs...")
        # Fetch all product IDs created in this session more reliably
        # This assumes IDs are sequential and new ones are higher.
        # For a truly robust ID fetch after executemany, specific DB features or more complex queries are needed.
        # For now, let's get all IDs and assume they match the number generated.
        # A better way is to fetch based on `created_at` or a session marker if possible.
        
        # Clear Faker's unique cache if it was used extensively, for next potential runs
        if hasattr(fake.unique, 'clear'):
            fake.unique.clear()

        # Fetch IDs of products inserted. For simplicity, we get all product IDs.
        # If running this script multiple times, this will fetch ALL product IDs in the table.
        # For precise IDs from THIS run, more complex logic (e.g., created_at range, or temp table) would be needed.
        # Given the auto_increment nature, we can assume the last N IDs are the ones we added.
        cursor.execute(f"SELECT id FROM Product ORDER BY id DESC LIMIT {NUM_PRODUCTS_TO_GENERATE}")
        fetched_ids = cursor.fetchall()
        inserted_product_ids = [row[0] for row in fetched_ids]
        inserted_product_ids.reverse() # To keep original order if that matters
        total_products_actually_inserted = len(inserted_product_ids)
        print(f"Fetched {total_products_actually_inserted} product IDs for variant generation.")

        if not inserted_product_ids:
            print("No product IDs found or fetched. Cannot generate variants. Exiting.")
            return

        # --- 2. Generate and Insert Product Variants ---
        print(f"\nGenerating and inserting product variants for {total_products_actually_inserted} products...")
        variant_data_batch = []
        variants_to_insert_count = 0

        sql_variant = "INSERT INTO Product_Variant (product_id, color, size, price, stock_qty) VALUES (%s, %s, %s, %s, %s)"

        for product_id in inserted_product_ids:
            num_variants = random.randint(1, MAX_VARIANTS_PER_PRODUCT)
            for _ in range(num_variants):
                color = random.choice(COLORS)
                size = random.choice(SIZES)
                price = round(random.uniform(5.0, 750.0), 2) 
                stock_qty = random.randint(0, 1000) 

                variant_data_batch.append((product_id, color, size, price, stock_qty))
                variants_to_insert_count += 1

                if len(variant_data_batch) >= BATCH_SIZE:
                    print(f"  Inserting batch of {len(variant_data_batch)} variants (total variants processed for batching: {variants_to_insert_count})...")
                    cursor.executemany(sql_variant, variant_data_batch)
                    cnx.commit()
                    total_variants_actually_inserted += len(variant_data_batch)
                    variant_data_batch = [] # Clear the batch

        # Insert any remaining variants in the last batch
        if variant_data_batch:
            print(f"  Inserting remaining {len(variant_data_batch)} variants (total variants processed for batching: {variants_to_insert_count})...")
            cursor.executemany(sql_variant, variant_data_batch)
            cnx.commit()
            total_variants_actually_inserted += len(variant_data_batch)

        print(f"\nData generation and insertion complete!")
        print(f"Total products actually inserted/used for variants: {total_products_actually_inserted}")
        print(f"Total variants inserted: {total_variants_actually_inserted}")

    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")
        if cnx and cnx.is_connected():
            try:
                cnx.rollback() # Rollback any uncommitted changes on error
                print("Transaction rolled back.")
            except mysql.connector.Error as roll_err:
                print(f"Error during rollback: {roll_err}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if cursor:
            cursor.close()
        if cnx and cnx.is_connected():
            cnx.close()
            print("Database connection closed.")

## 5. Execute Data Generation

Run the cell below to start the data generation process.

**Note:** Ensure your `DB_CONFIG` is correctly set up before running. If you run this multiple times, it will add more data to your tables. You might want to clear the tables manually if you need a fresh start for testing.

In [14]:
if __name__ == "__main__" or __name__ == "__mp_main__": # Check for notebook environment too
    # --- IMPORTANT ---
    # Before running, make sure to update the DB_CONFIG cell with your MySQL credentials
    # and the database name. 
    # Also, ensure your tables are created in the database.
    # ---
    print("Starting mock data generation script...")
    
    # Check if DB_CONFIG is still placeholder
    if DB_CONFIG['user'] == 'your_user' or DB_CONFIG['database'] == 'your_database':
        print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! PLEASE UPDATE DB_CONFIG WITH YOUR ACTUAL DATABASE DETAILS BEFORE RUNNING !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")
    else:
        generate_and_insert_mock_data()
        
    print("Script finished.")

Starting mock data generation script...
Successfully connected to the database: test_database

Generating and inserting 100 products...
  Inserting batch of 50 products (total attempted: 50/100)...
  Inserting batch of 50 products (total attempted: 100/100)...
All products batches sent. Fetching all inserted product IDs...
Fetched 100 product IDs for variant generation.

Generating and inserting product variants for 100 products...
  Inserting batch of 50 variants (total variants processed for batching: 50)...
  Inserting batch of 50 variants (total variants processed for batching: 100)...
  Inserting batch of 50 variants (total variants processed for batching: 150)...
  Inserting batch of 50 variants (total variants processed for batching: 200)...
  Inserting batch of 50 variants (total variants processed for batching: 250)...
  Inserting remaining 40 variants (total variants processed for batching: 290)...

Data generation and insertion complete!
Total products actually inserted/used